In [80]:
import numpy as np
import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score

# from sklearn.model_selection import KFold # 交差検証法に関する関数

import warnings
warnings.simplefilter('ignore')


## データの読み込み

In [81]:
# データの読み込み
# df_org = pd.read_csv('./data/input_01.csv')
# df_org = pd.read_csv('./data/input_02.csv')
df_org = pd.read_csv('./data/input_03.csv')

In [82]:
# データ量が多いのでコーディング中はサンプリング
# df = df_org.sample(100000, replace=True)
df = df_org

## データの準備

In [83]:
# 数値エンコーディング（GBDT以外の場合は必須）
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df = df.apply(encoder.fit_transform)

In [84]:
y = df[['state']]
# x_cols = ['category', 'main_category', 'currency', 'country', 'period', 'n_words', 'log_goal']
# x_cols = ['class', 'currency', 'country', 'period', 'n_words', 'log_goal']
# x_cols = ['class', 'main_category', 'currency', 'country', 'period', 'n_words', 'log_goal']
x_cols = ['class', 'currency', 'country', 'period', 'n_words', 'log_goal', 'launched_year', 'launched_month', 'launched_day']
x = df[x_cols]

In [85]:
# y.shape

In [86]:
x.head()

,class,currency,country,period,n_words,log_goal,launched_year,launched_month,launched_day
0,97,5,9,58,5,10270,6,7,10
1,42,13,22,59,7,35206,8,8,1
2,42,13,22,44,2,37850,4,0,11
3,78,13,22,29,6,19451,3,2,16
4,55,13,22,34,2,38482,7,1,25


## モデル学習・推論

In [93]:
df.head()

,Unnamed: 0,ID,category,main_category,currency,goal,pledged,state,backers,country,usd pledged,usd_pledged_real,period,launched_year,launched_month,launched_day,class,n_words,log_goal
0,0,154087,108,12,5,689,0,0,0,9,0,0,58,6,7,10,97,5,10270
1,1,154088,93,6,13,5532,8330,0,15,22,6867,34063,59,8,8,1,42,7,35206
2,2,154089,93,6,13,5994,1094,0,3,22,11452,10300,44,4,0,11,42,2,37850
3,3,154090,90,10,13,2405,1,0,1,22,51,53,29,3,2,16,78,6,19451
4,4,154091,123,7,13,6124,49937,1,224,22,82385,89274,34,7,1,25,55,2,38482


In [106]:
from sklearn.model_selection import train_test_split
# ランダムシード値
RANDOM_STATE = 10

# 学習データと評価データの割合
TEST_SIZE = 0.2

# 学習データと評価データを作成
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=TEST_SIZE,random_state=RANDOM_STATE)

# 学習データと評価データを作成
x_train, x_valid, y_train, y_valid = train_test_split(x_train,y_train,test_size=TEST_SIZE,random_state=RANDOM_STATE)

lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_valid, y_valid, reference=lgb_train)

In [107]:
# LightGBMのパラメータ設定
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_leaves': 50,
    'verbose': 0
}

In [109]:
# LightGBM学習
model = lgb.train(params, lgb_train, num_round, 
                  valid_names=['train', 'valid'], valid_sets=lgb_eval,
                  early_stopping_rounds=20, verbose_eval=False)

# LightGBM推論
y_pred = model.predict(x_test, num_iteration=model.best_iteration)

# 評価
y_pred = y_pred.round(0)
score_valid = accuracy_score(y_test, y_pred)
print(f"{itr:02}test_accuracy_score: {score_valid:.4f} %")    

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004978 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
06test_accuracy_score: 0.6980 %


## ハイパーパラメータ最適化

In [110]:
# LightGBMのパラメータ設定
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_leaves': 50,
    'verbose': 0
}

In [111]:
# ランダムシード値
RANDOM_STATE = 10

# 学習データと評価データの割合
TEST_SIZE = 0.2

# 学習データと評価データを作成
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=TEST_SIZE,random_state=RANDOM_STATE)

# 学習データと評価データを作成
x_train, x_valid, y_train, y_valid = train_test_split(x_train,y_train,test_size=TEST_SIZE,random_state=RANDOM_STATE)

lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_valid, y_valid, reference=lgb_train)

In [113]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.2/308.2 kB 3.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.0/81.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.1/147.1 kB 15.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11123 sha256=912c9d6e12dce62c80f9766b705a344582c6598f7a86f91dcd352476ffb4e70d
  Stored in directory: /home/jovyan/.cache/pip/wheels/04/24/fe/140a94a7f1036003ede94579e6b4227fe96c840c6f4dcbe307
Successfully built pyperclip


In [114]:
# LightGBM
import optuna.integration.lightgbm as lgb

# ベストなパラメータ、途中経過を保存する
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    "verbosity": -1,
    "boosting_type": "gbdt",
}

best_params, history = {}, []

# LightGBM学習
model = lgb.train(params,
                lgb_train,
                num_boost_round=200,
                valid_sets=[lgb_train, lgb_eval],
                early_stopping_rounds=50
               )

best_params = model.params
best_params


[I 2022-07-24 14:11:57,153] A new study created in memory with name: no-name-b80b58f5-aba6-40e4-9fc1-25dceeabdbf1
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

[1]	valid_0's binary_logloss: 0.665575	valid_1's binary_logloss: 0.66636
Training until validation scores don't improve for 50 rounds
[2]	valid_0's binary_logloss: 0.657961	valid_1's binary_logloss: 0.658639
[3]	valid_0's binary_logloss: 0.651344	valid_1's binary_logloss: 0.651899
[4]	valid_0's binary_logloss: 0.645518	valid_1's binary_logloss: 0.646051
[5]	valid_0's binary_logloss: 0.64031	valid_1's binary_logloss: 0.640799
[6]	valid_0's binary_logloss: 0.635513	valid_1's binary_logloss: 0.636003
[7]	valid_0's binary_logloss: 0.630831	valid_1's binary_logloss: 0.631355
[8]	valid_0's binary_logloss: 0.627333	valid_1's binary_logloss: 0.627821
[9]	valid_0's binary_logloss: 0.624225	valid_1's binary_logloss: 0.62468
[10]	valid_0's binary_logloss: 0.621558	valid_1's binary_logloss: 0.621849
[11]	valid_0's binary_logloss: 0.618985	valid_1's binary_logloss: 0.619198
[12]	valid_0's binary_logloss: 0.616377	valid_1's binary_logloss: 0.616602
[13]	valid_0's binary_logloss: 0.61422	valid_1's bi

feature_fraction, val_score: 0.567734:  14%|#4        | 1/7 [00:02<00:15,  2.51s/it]

[175]	valid_0's binary_logloss: 0.559009	valid_1's binary_logloss: 0.568164
[176]	valid_0's binary_logloss: 0.558904	valid_1's binary_logloss: 0.568115
[177]	valid_0's binary_logloss: 0.55878	valid_1's binary_logloss: 0.568052
[178]	valid_0's binary_logloss: 0.55864	valid_1's binary_logloss: 0.567973
[179]	valid_0's binary_logloss: 0.558546	valid_1's binary_logloss: 0.567948
[180]	valid_0's binary_logloss: 0.558478	valid_1's binary_logloss: 0.567942
[181]	valid_0's binary_logloss: 0.55842	valid_1's binary_logloss: 0.567941
[182]	valid_0's binary_logloss: 0.558366	valid_1's binary_logloss: 0.567933
[183]	valid_0's binary_logloss: 0.558259	valid_1's binary_logloss: 0.567899
[184]	valid_0's binary_logloss: 0.558165	valid_1's binary_logloss: 0.567841
[185]	valid_0's binary_logloss: 0.558086	valid_1's binary_logloss: 0.567829
[186]	valid_0's binary_logloss: 0.558015	valid_1's binary_logloss: 0.567802
[187]	valid_0's binary_logloss: 0.557937	valid_1's binary_logloss: 0.5678
[188]	valid_0's b

feature_fraction, val_score: 0.567621:  29%|##8       | 2/7 [00:05<00:12,  2.56s/it]

...

num_leaves, val_score: 0.564143:  75%|#######5  | 15/20 [00:49<00:19,  3.80s/it]

[1]	valid_0's binary_logloss: 0.667582	valid_1's binary_logloss: 0.668605
Training until validation scores don't improve for 50 rounds
[2]	valid_0's binary_logloss: 0.655021	valid_1's binary_logloss: 0.656266
[3]	valid_0's binary_logloss: 0.645602	valid_1's binary_logloss: 0.646775
[4]	valid_0's binary_logloss: 0.637844	valid_1's binary_logloss: 0.639451
[5]	valid_0's binary_logloss: 0.632086	valid_1's binary_logloss: 0.634057
[6]	valid_0's binary_logloss: 0.623881	valid_1's binary_logloss: 0.626001
[7]	valid_0's binary_logloss: 0.61736	valid_1's binary_logloss: 0.619751
[8]	valid_0's binary_logloss: 0.613114	valid_1's binary_logloss: 0.615868
[9]	valid_0's binary_logloss: 0.609832	valid_1's binary_logloss: 0.612932
[10]	valid_0's binary_logloss: 0.605338	valid_1's binary_logloss: 0.608719
[11]	valid_0's binary_logloss: 0.603163	valid_1's binary_logloss: 0.606773
[12]	valid_0's binary_logloss: 0.598676	valid_1's binary_logloss: 0.602479
[13]	valid_0's binary_logloss: 0.594666	valid_1's

num_leaves, val_score: 0.564143:  80%|########  | 16/20 [00:53<00:15,  3.77s/it]

[1]	valid_0's binary_logloss: 0.668074	valid_1's binary_logloss: 0.668895
Training until validation scores don't improve for 50 rounds
[2]	valid_0's binary_logloss: 0.656705	valid_1's binary_logloss: 0.657473
[3]	valid_0's binary_logloss: 0.64842	valid_1's binary_logloss: 0.649026
[4]	valid_0's binary_logloss: 0.641204	valid_1's binary_logloss: 0.64197
[5]	valid_0's binary_logloss: 0.635751	valid_1's binary_logloss: 0.636661
[6]	valid_0's binary_logloss: 0.628513	valid_1's binary_logloss: 0.629459
[7]	valid_0's binary_logloss: 0.622917	valid_1's binary_logloss: 0.624078
[8]	valid_0's binary_logloss: 0.618975	valid_1's binary_logloss: 0.620215
[9]	valid_0's binary_logloss: 0.615899	valid_1's binary_logloss: 0.617288
[10]	valid_0's binary_logloss: 0.61123	valid_1's binary_logloss: 0.612691
[11]	valid_0's binary_logloss: 0.609227	valid_1's binary_logloss: 0.610704
[12]	valid_0's binary_logloss: 0.605168	valid_1's binary_logloss: 0.606686
[13]	valid_0's binary_logloss: 0.601677	valid_1's b

KeyboardInterrupt: 

In [ ]:

from sklearn.metrics import log_loss, roc_auc_score, accuracy_score
import lightgbm as lgb

# scores_train = []
scores_valid = []
itr = 1

for train_idx, valid_idx in kf.split(x, y):
    x_train, y_train = x.iloc[train_idx], y.iloc[train_idx]
    x_valid, y_valid = x.iloc[valid_idx], y.iloc[valid_idx]
    
    # lightgbmのデータ構造に変換
    lgb_train = lgb.Dataset(x_train, y_train)
    lgb_valid = lgb.Dataset(x_valid, y_valid)
    
    # ハイパーパラメータの設定
    # metricsはaccuracy_scoreにすべきかも
    params = {'num_leaves': 64, 'objective': 'binary', 'seed': seed, 
              'verbose': 0, 'metrics': 'binary_logloss'}
    num_round = 1000

    # 学習の実行
    # カテゴリ変数を指定した場合の挙動はKaggle本 p240参照
    # categorical_features = ['category', 'main_category', 'currency', 'country']
    model = lgb.train(params, lgb_train, num_round, 
                    valid_names=['train', 'valid'], valid_sets=[lgb_train, lgb_valid],
                     early_stopping_rounds=20, verbose_eval=False)
    
    # # 訓練データに対する予測、正答率
    # y_pred = clf.predict(x_train)
    # acc_train = accuracy_score(y_true=y_train, y_pred=y_pred)
    # print(f"{itr:02}train_訓練データの正答率: {100 * acc_train:.2f} %")
    # scores_train.append(acc_train)  
    
    # 検証データに対する予測
    y_pred = model.predict(x_valid)
    # accurasy_scoreするには0で丸め込みが必要（https://shiokoji11235.com/two_interface_of_lightgbm）
    y_pred = y_pred.round(0)
    # score_valid = log_loss(y_true=y_valid, y_pred=y_pred)
    score_valid = accuracy_score(y_valid, y_pred)
    # score_valid = roc_auc_score(y_valid, y_pred)
    # print(f"{itr:02}valid_logloss: {score_valid:.4f} %")    
    print(f"{itr:02}valid_accuracy_score: {score_valid:.4f} %")    
    # print(f"{itr:02}valid_auc_score: {score_valid:.4f} %")    
    scores_valid.append(score_valid)
    
    itr += 1

## 交差検証

In [87]:
# !pip install lightgbm

In [88]:
from sklearn.model_selection import KFold
n_split = 5 # グループ数を設定（今回は5分割）
seed = 1234
# cross_valid_mae = 0
split_num = 1
kf = KFold(n_splits=n_split, shuffle=True, random_state=seed)

In [89]:
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score
import lightgbm as lgb

# scores_train = []
scores_valid = []
itr = 1

for train_idx, valid_idx in kf.split(x, y):
    x_train, y_train = x.iloc[train_idx], y.iloc[train_idx]
    x_valid, y_valid = x.iloc[valid_idx], y.iloc[valid_idx]
    
    # lightgbmのデータ構造に変換
    lgb_train = lgb.Dataset(x_train, y_train)
    lgb_valid = lgb.Dataset(x_valid, y_valid)
    
    # ハイパーパラメータの設定
    # metricsはaccuracy_scoreにすべきかも
    params = {'num_leaves': 64, 'objective': 'binary', 'seed': seed, 
              'verbose': 0, 'metrics': 'binary_logloss'}
    num_round = 1000

    # 学習の実行
    # カテゴリ変数を指定した場合の挙動はKaggle本 p240参照
    # categorical_features = ['category', 'main_category', 'currency', 'country']
    model = lgb.train(params, lgb_train, num_round, 
                    valid_names=['train', 'valid'], valid_sets=[lgb_train, lgb_valid],
                     early_stopping_rounds=20, verbose_eval=False)
    
    # # 訓練データに対する予測、正答率
    # y_pred = clf.predict(x_train)
    # acc_train = accuracy_score(y_true=y_train, y_pred=y_pred)
    # print(f"{itr:02}train_訓練データの正答率: {100 * acc_train:.2f} %")
    # scores_train.append(acc_train)  
    
    # 検証データに対する予測
    y_pred = model.predict(x_valid)
    # accurasy_scoreするには0で丸め込みが必要（https://shiokoji11235.com/two_interface_of_lightgbm）
    y_pred = y_pred.round(0)
    # score_valid = log_loss(y_true=y_valid, y_pred=y_pred)
    score_valid = accuracy_score(y_valid, y_pred)
    # score_valid = roc_auc_score(y_valid, y_pred)
    # print(f"{itr:02}valid_logloss: {score_valid:.4f} %")    
    print(f"{itr:02}valid_accuracy_score: {score_valid:.4f} %")    
    # print(f"{itr:02}valid_auc_score: {score_valid:.4f} %")    
    scores_valid.append(score_valid)
    
    itr += 1

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006557 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
01valid_accuracy_score: 0.6979 %
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004476 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
02valid_accuracy_score: 0.6958 %
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004724 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
03valid_accuracy_score: 0.7014 %
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004942 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_c

In [90]:
# 各foldのスコアの平均
# print(np.mean(scores_train))
print(np.mean(scores_valid))

0.6980055777493028


### input_01.csv
- 検証データに対する正答率：0.69899


### input_02.csv（class, main_category）
- 検証データに対する正答率：0.70896


### input_02.csv（class）
- 検証データに対する正答率：0.70765


### input_03.csv（class, yar/month/day）
- 検証データに対する正答率：0.7159


## テストデータに対する予測
本来はtrain/validではなくtrain/valid/testに分割し、testに対する予測を見るべき